# Capstone Project - The Battle of the Neighborhoods in Dortmund

## 1. Introduction/Business Understanding

### 1.1 Description of the problem

Our business problem we are facing is that around every two to three weeks there are football home games of the First Bundesliga Club Borussia Dortmund. How can we provide support to visitors to list and visualize Dortmund boroughs that fit their needs for food venues.

### 1.2 Discussion of the background

Dortmund is a city with around 585.000 inhabitants and twelve boroughs. Dortmund is well known for its football First Bundesliga Club Borussia Dortmund. Many visitors come to Dortmund to watch their games.

I think it is hard vor visitors to choose the right place to stay, especially if there are looking for lots and good food choices. As there are many recommendations on the web it's hard to filter out which is the right choice for the visitor.

In this capstone project I am adressing this problem with the help of Foursquare location data and machine learning to help the visitors to make a decision and find the appropriate neighborhood. I will use the Foursquare location data and clustering methods for grouping the boroughs to different groups by their restaurant venues information.

## 2. Data

We will use the following data:

**List of the twelve boroughs of Dortmund with their latitude and longitude**

* Datasource: https://de.wikipedia.org/wiki/Liste_der_Dortmunder_Stadtteile

* Description: We will use the first list and drop the details so that we have only the twelve boroughs and enrich the table with the coordinates using geocoder class of Geopy client.

**Restaurants in each borough**

* Data source: Foursquare APIs

* Description : By this we will get all the venues in each borough. We will filter these venues to get only restaurants.